In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from operator import itemgetter
import random
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import choice
import pandas as pd

In [ ]:
txt = open("text.txt", "r")
tokens = [t.lower() for t in word_tokenize(txt.read())] #splitting text into tokens and case-folding to lowercase

In [ ]:
def generate_initial_population(population_size): # Function which generates an n number of keyboards with key positions randomised
    key_placements = []
    for _ in range(population_size):
        key_placement = {}
        chars = "qazwsxedcrfvtgbyhnujmik,ol.p;?"

        placements = [(0,0), (0.4,1), (0.8,2),
                      (1,0), (1.4,1), (1.8,2),
                      (2,0), (2.4,1), (2.8,2),
                      (3,0), (3.4,1), (3.8,2),
                      (4,0), (4.4,1), (4.8,2),
                      (5,0), (5.4,1), (5.8,2),
                      (6,0), (6.4,1), (6.8,2),
                      (7,0), (7.4,1), (7.8,2),
                      (8,0), (8.4,1), (8.8,2),
                      (9,0), (9.4,1), (9.8,2)]
        
        for char in chars:
            placement = random.choice(placements)
            placements.remove(placement) # Removing position so that it cannot be chosen again
            key_placement[char] = placement
        key_placements.append(key_placement)
    return key_placements

In [ ]:
def calculate_fitness(keyboard_list, text): # Function that calculates the total distance required to type a give text for each keyboard layout providied
    keyboard_distances = {}
    for i, keyboard in enumerate(keyboard_list):
        # Defining the key which each finger will originally be placed on
        f1 = list(keyboard.keys()) [list(keyboard.values()).index((1.4, 1))]
        f2 = list(keyboard.keys()) [list(keyboard.values()).index((2.4, 1))]
        f3 = list(keyboard.keys()) [list(keyboard.values()).index((3.4, 1))]
        f4 = list(keyboard.keys()) [list(keyboard.values()).index((6.4, 1))]
        f5 = list(keyboard.keys()) [list(keyboard.values()).index((7.4, 1))]
        f6 = list(keyboard.keys()) [list(keyboard.values()).index((8.4, 1))]

        d = 0
        
        chars = [i for ele in tokens for i in ele]
        
        for char in chars:
            if (char.isascii() and char.isalpha()) or char == ';' or char == ',' or char == '.' or char == '?': # isascii() makes sure that no non-english letters are considered while isalpha() makes sure that no numerical values are considered
                key_location = keyboard[char]
                # Defining the positions whci heach finger is responsible for typing
                if key_location == (0, 0) or key_location == (0.3, 1) or key_location == (0.8, 2) or key_location == (1, 0) or key_location == (1.3, 1) or key_location == (1.8, 2):
                    d += math.dist(key_location, keyboard[f1])
                    f1 = char
                elif key_location == (2, 0) or key_location == (2.3, 1) or key_location == (2.8, 2):
                    d += math.dist(key_location, keyboard[f2])
                    f2 = char
                elif key_location == (3, 0) or key_location == (3.3, 1) or key_location == (3.8, 2) or key_location == (4, 0) or key_location == (4.3, 1) or key_location == (4.8, 2):
                    d += math.dist(key_location, keyboard[f3])
                    f3 = char
                elif key_location == (5, 0) or key_location == (5.3, 1) or key_location == (5.8, 2) or key_location == (6, 0) or key_location == (6.3, 1) or key_location == (6.8, 2):
                    d += math.dist(key_location, keyboard[f4])
                    f4 = char
                elif key_location == (7, 0) or key_location == (7.3, 1) or key_location == (7.8, 2):
                    d += math.dist(key_location, keyboard[f5])
                    f5 = char
                elif key_location == (8, 0) or key_location == (8.3, 1) or key_location == (8.8, 2) or key_location == (9, 0) or key_location == (9.3, 1) or key_location == (9.8, 2):
                    d += math.dist(key_location, keyboard[f6])
                    f6 = char
            else: pass # If character is not one of those we are checking for, ignore it
        keyboard_distances[i] = d
    
    return keyboard_distances

In [ ]:
def elitism(population, keyboard_distances):
    best_keyboards = []
    
    values = sorted(keyboard_distances.values()) # Sorting fitness values in ascending order
    
    index1 = list(keyboard_distances.keys())[list(keyboard_distances.values()).index(values[0])] # Obtaining the index of the keyboard with the best (lowest) fitness value
    
    # Using a for loop to make sure that the second keyboard returned is not the exact same as the first
    for value in values[1:]:
        if value > values[0]:
            index2 = list(keyboard_distances.keys())[list(keyboard_distances.values()).index(value)]
            break

    best_keyboards.append(population[index1])
    best_keyboards.append(population[index2])

    return best_keyboards

In [ ]:
def select_keyboards(population, keyboard_distances): # Selection function which chooses 2 keyboards from the top 50% of keyboards of the given population
    selected_keyboards = []
    
    # Sorting the dictionary to identify the best keyboards
    keys = list(keyboard_distances.keys())
    values = list(keyboard_distances.values())
    sorted_value_index = np.argsort(values)
    sorted_dict = {keys[i]: values[i] for i in sorted_value_index}
    
    keyboards = []
    
    # Creating a list of keyboards which will be sorted from best fitness to worst
    for i in sorted_dict.keys():
        keyboards.append(population[i])
        
    length = len(sorted_dict)
    middle_index = length//2

    # Obtaining the first half of the list which will be the top 50% of keyboards with the best fitness score
    first_half = keyboards[:middle_index]
    
    # Rank Based Selection:
    
    n = int(len(population) / 2)
    
    rank_sum = n * (n + 1) / 2 # Gauss Summation
    
    probability = []
    
    # Calculating the probabilty of each chromosome to be seletced
    for i in list(range(n,0,-1)): # For loop which starts from n and decrements i by 1 until i = 1
        probability.append(i/rank_sum)
    
    selected_keyboards = choice(first_half, 2, replace = False, p = probability)

    return selected_keyboards

In [ ]:
def crossover(parent_keyboard1, parent_keyboard2):
    offset_keyboards = []
    
    # Two Point Crossover
    
    offset_parent_keyboard1 = {}
    offset_parent_keyboard2 = {}
    
    placements = [(0,0), (0.4,1), (0.8,2),
                  (1,0), (1.4,1), (1.8,2),
                  (2,0), (2.4,1), (2.8,2),
                  (3,0), (3.4,1), (3.8,2),
                  (4,0), (4.4,1), (4.8,2),
                  (5,0), (5.4,1), (5.8,2),
                  (6,0), (6.4,1), (6.8,2),
                  (7,0), (7.4,1), (7.8,2),
                  (8,0), (8.4,1), (8.8,2),
                  (9,0), (9.4,1), (9.8,2)]
    
    # 2 splitpoints are chosen - 1 from the left side of the keyboard and the other from the right side
    
    placements_left = [(0,0), (0.4,1), (0.8,2),
                       (1,0), (1.4,1), (1.8,2),
                       (2,0), (2.4,1), (2.8,2),
                       (3,0), (3.4,1), (3.8,2),
                       (4,0), (4.4,1), (4.8,2)]
    
    placements_right = [(5,0), (5.4,1), (5.8,2),
                        (6,0), (6.4,1), (6.8,2),
                        (7,0), (7.4,1), (7.8,2),
                        (8,0), (8.4,1), (8.8,2),
                        (9,0), (9.4,1), (9.8,2)]
    
    splitpoint_left = random.choice(placements_left)
    splitpoint_right = random.choice(placements_right)
    
    indexleft = placements_left.index(splitpoint_left)
    
    # All keys to the left of the left splitpoint are inserted into the offset keyboard
    # at the position in which they were located in the parent keyboard
    
    for key in parent_keyboard1.keys():
        if parent_keyboard1[key] in placements_left[:indexleft]:
            offset_parent_keyboard1[key] = parent_keyboard1.get(key)
            
    for key in parent_keyboard2.keys():
        if parent_keyboard2[key] in placements_left[:indexleft]:
            offset_parent_keyboard2[key] = parent_keyboard2.get(key)

    indexright = placements_right.index(splitpoint_right)
    
    # All keys at or to the right of the right splitpoint are inserted into the offset keyboard
    # at the position in which they were located in the parent keyboard
    
    for key in parent_keyboard1.keys():
        if parent_keyboard1[key] in placements_right[indexright:]:
            offset_parent_keyboard1[key] = parent_keyboard1.get(key)
            
    for key in parent_keyboard2.keys():
        if parent_keyboard2[key] in placements_right[indexright:]:
            offset_parent_keyboard2[key] = parent_keyboard2.get(key)
    
    # Defining the order in which the parent keyboards will be traveresd in order to fill in the rest of the offset keyboards
    order = placements[indexleft+1:] + placements[:indexleft+1]
    
    # Keys are instsrted into the offset keyboard depending on the position in which they appear after the first splitpoint
    for pos in order:
        keys = list(parent_keyboard2.keys())
        values = list(parent_keyboard2.values())
        
        position = values.index(pos)
        key = keys[position]
        
        if len(offset_parent_keyboard1.values()) == 30: break
        
        elif key not in offset_parent_keyboard1.keys():
            for i in placements:
                if i not in offset_parent_keyboard1.values():
                    offset_parent_keyboard1[key] = i
                    break
    offset_keyboards.append(offset_parent_keyboard1)
            
    for pos in order:
        keys = list(parent_keyboard1.keys())
        values = list(parent_keyboard1.values())
        
        position = values.index(pos)
        key = keys[position]
        
        if len(offset_parent_keyboard2.values()) == 30: break
        
        elif key not in offset_parent_keyboard2.keys():
            for i in placements:
                if i not in offset_parent_keyboard2.values():
                    offset_parent_keyboard2[key] = i
                    break
    offset_keyboards.append(offset_parent_keyboard2)
    
    return offset_keyboards

In [ ]:
def run_evolution():
    population = generate_initial_population(10)
    number_of_generations = 500
    low = []
    gens = []
    avg = []
    for i in range(number_of_generations):
        gens.append(i)
        print("Generation {}:".format(i+1))
        print("")
        keyboard_distances = calculate_fitness(population, tokens)
        print(keyboard_distances)
        print("")
        avg.append(sum(keyboard_distances.values()) / 10)
        print(sum(keyboard_distances.values()) / 10)
        print("")
        lowest = min(keyboard_distances.values())
        low.append(lowest)
        next_generation = elitism(population, keyboard_distances)
        
        for j in range(int(len(population) / 2) - 1):
            parents = select_keyboards(population, keyboard_distances)
            new_keyboards = crossover(parents[0], parents[1])
            next_generation += new_keyboards
            
        population = next_generation
        
    %matplotlib qt
    plt.figure(1)
    plt.plot(gens, low, c = "red", linewidth = 2)
    plt.title("Lowest Distance")
    plt.xlabel("Generation")
    plt.ylabel("Distance")
    plt.show()
    
    plt.figure(2)
    plt.plot(gens, avg, c = "red", linewidth = 2)
    plt.title("Average Distance")
    plt.xlabel("Generation")
    plt.ylabel("Distance")
    plt.show()
    
    return population

In [ ]:
run_evolution()

In [ ]:
from collections import Counter

chars = [i for ele in tokens for i in ele]

output = Counter(chars)

print(output)

In [ ]:
def optimised():
    optimised_layout = {}
    
    placements = [(0,0), (0.4,1), (0.8,2),
                  (1,0), (1.4,1), (1.8,2),
                  (2,0), (2.4,1), (2.8,2),
                  (3,0), (3.4,1), (3.8,2),
                  (4,0), (4.4,1), (4.8,2),
                  (5,0), (5.4,1), (5.8,2),
                  (6,0), (6.4,1), (6.8,2),
                  (7,0), (7.4,1), (7.8,2),
                  (8,0), (8.4,1), (8.8,2),
                  (9,0), (9.4,1), (9.8,2)]
    
    chars = "ptvhrxfo.kdg,ibmazwl;qunjec?sy"
        
    for i, char in enumerate(chars):
        optimised_layout[char] = placements[i]
        
    return optimised_layout

In [ ]:
def qwerty():
    qwerty_layout = {}
    
    placements = [(0,0), (0.4,1), (0.8,2),
                  (1,0), (1.4,1), (1.8,2),
                  (2,0), (2.4,1), (2.8,2),
                  (3,0), (3.4,1), (3.8,2),
                  (4,0), (4.4,1), (4.8,2),
                  (5,0), (5.4,1), (5.8,2),
                  (6,0), (6.4,1), (6.8,2),
                  (7,0), (7.4,1), (7.8,2),
                  (8,0), (8.4,1), (8.8,2),
                  (9,0), (9.4,1), (9.8,2)]
    
    chars = "qazwsxedcrfvtgbyhnujmik,ol.p;?"
        
    for i, char in enumerate(chars):
        qwerty_layout[char] = placements[i]
        
    return qwerty_layout

In [ ]:
def azerty():
    azerty_layout = {}
    
    placements = [(0,0), (0.4,1), (0.8,2),
                  (1,0), (1.4,1), (1.8,2),
                  (2,0), (2.4,1), (2.8,2),
                  (3,0), (3.4,1), (3.8,2),
                  (4,0), (4.4,1), (4.8,2),
                  (5,0), (5.4,1), (5.8,2),
                  (6,0), (6.4,1), (6.8,2),
                  (7,0), (7.4,1), (7.8,2),
                  (8,0), (8.4,1), (8.8,2),
                  (9,0), (9.4,1), (9.8,2)]
    
    chars = "aqwzsxedcrfvtgbyhnuj,ik;ol.pm?"

    for i, char in enumerate(chars):
        azerty_layout[char] = placements[i]
        
    return azerty_layout

In [ ]:
def dvorak():
    dvorak_layout = {}
    
    placements = [(0,0), (0.4,1), (0.8,2),
                  (1,0), (1.4,1), (1.8,2),
                  (2,0), (2.4,1), (2.8,2),
                  (3,0), (3.4,1), (3.8,2),
                  (4,0), (4.4,1), (4.8,2),
                  (5,0), (5.4,1), (5.8,2),
                  (6,0), (6.4,1), (6.8,2),
                  (7,0), (7.4,1), (7.8,2),
                  (8,0), (8.4,1), (8.8,2),
                  (9,0), (9.4,1), (9.8,2)]
    
    chars = "?a;,oq.ejpukyixfdbghmctwrnvlsz"

    for i, char in enumerate(chars):
        dvorak_layout[char] = placements[i]
        
    return dvorak_layout

In [ ]:
keyboard_list = []

keyboard_list.append(qwerty())
keyboard_list.append(azerty())
keyboard_list.append(dvorak())
keyboard_list.append(optimised())

keyboard_distances = calculate_fitness(keyboard_list, tokens)

layouts = ["QWERTY", "AZERTY", "DVORAK", "RODLUE"]
values = keyboard_distances.values()
print(values)


plt.bar(layouts, values, color = "red")